## Importa funções para o pré-processamento

In [3]:
from pre_processamento import *

from dlisio import dlis
import pandas as pd

# A função 'glob' do módulo 'glob' é usada para procurar todos os arquivos em um diretório com determinada extensão
import glob

## Carrega os dados

In [4]:
nomes_arquivos = []     # Armazena os nomes dos arquivos .dlis
leituras_dlis = []      # Armazena as leituras dos arquivos .dlis
nomes_anp = []          # Armazena os nomes obtidos das leituras

for file in glob.glob(r'**/Data' + "/*.dlis", recursive=True):
    # Salva o nome do arquivo
    nomes_arquivos.append(file)

    # Salva os dados da leitura
    leitura, *tail = dlis.load(f'{file}')
    leituras_dlis.append(leitura)

    # Salva o nome do poço
    nome = leitura.origins[0].well_name
    nomes_anp.append(nome)

In [5]:
print(nomes_arquivos)
print(nomes_anp)

['Data\\1-brsa-551-se_brsa_raw.dlis', 'Data\\1-brsa-574-se_brsa_raw.dlis', 'Data\\1-brsa-595-se_brsa_raw-1.dlis', 'Data\\1-brsa-605-se_brsa_raw.dlis', 'Data\\1-brsa-659-se_brsa_raw.dlis', 'Data\\3-CP-1847-SE.dlis', 'Data\\3-CP-1848-SE.dlis', 'Data\\3-CP-1851-SE.dlis', 'Data\\3-CP-1853-SE.dlis', 'Data\\3-CP-1855-SE.dlis', 'Data\\3-CP-1857-SE.dlis']
['1-FSG-1-SE', '1-FSJQ-1-SE', '1-BRSA-595-SE', '1-BRSA-605-SE', '1-BRSA-659-SE', '3-BRSA-900-SE', '3-BRSA-889-SE', '3-BRSA-912-SE', '3-BRSA-897-SE', '3-BRSA-910-SE', '3-BRSA-907-SE']


## Padroniza nomes fora do padão 'X-BRSA-XXX-SE'

In [6]:
nomes_anp[0] = '1-BRSA-551-SE'
nomes_anp[1] = '1-BRSA-574-SE'

In [7]:
nomes_anp

['1-BRSA-551-SE',
 '1-BRSA-574-SE',
 '1-BRSA-595-SE',
 '1-BRSA-605-SE',
 '1-BRSA-659-SE',
 '3-BRSA-900-SE',
 '3-BRSA-889-SE',
 '3-BRSA-912-SE',
 '3-BRSA-897-SE',
 '3-BRSA-910-SE',
 '3-BRSA-907-SE']

## Salva os identificadores em 'nomes_anp_abreviados'

In [8]:
nomes_anp_abreviados = []   # Armazena os nomes abreviados (identificadores)

for nome in nomes_anp:
    nome_abreviado = nome[7:10]
    nomes_anp_abreviados.append(nome_abreviado)

In [9]:
nomes_anp_abreviados

['551', '574', '595', '605', '659', '900', '889', '912', '897', '910', '907']

## Cria dicionário para armazenar os dados e respectivos nomes

In [10]:
# Casa itens da lista 'nome_anp_abreviados' com os itens da lista 'leituras_dlis'
pares = zip(nomes_anp_abreviados, leituras_dlis)

# Cria dicionário 'dli_dict'
dli_dict = dict(pares)
dli_dict

{'551': LogicalFile(AIT_SONIC_TLD_MCFL_018PUP),
 '574': LogicalFile(GEOLOAD.1),
 '595': LogicalFile(GEOLOAD.1),
 '605': LogicalFile(AIT_SONIC_TLD_MCFL_048PUC),
 '659': LogicalFile(GEOLOAD.1),
 '900': LogicalFile(GEOLOAD.1),
 '889': LogicalFile(GEOLOAD.1),
 '912': LogicalFile(GEOLOAD.1),
 '897': LogicalFile(GEOLOAD.1),
 '910': LogicalFile(GEOLOAD.1),
 '907': LogicalFile(GEOLOAD.1)}

## Separa TODAS as curvas presentes nos .dlis de cada poço

In [11]:
channels_dict = {}

for key, poco in dli_dict.items():
    channels = poco.frames[0].channels
    channels_list = [channel.name for channel in channels]
    channels_dict[key] = channels_list

## Salva as curvas presentes nos .dlis em arquivos CSV

In [12]:
import csv

for key, value in channels_dict.items():
    file_name = f"Curvas_CSV/curvas_{key}.csv"
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Curva", "Poco"])
        for item in value:
            writer.writerow([item, f'P_{key}'])

    print(f"Arquivo {file_name} criado com sucesso")

Arquivo Curvas_CSV/curvas_551.csv criado com sucesso
Arquivo Curvas_CSV/curvas_574.csv criado com sucesso
Arquivo Curvas_CSV/curvas_595.csv criado com sucesso
Arquivo Curvas_CSV/curvas_605.csv criado com sucesso
Arquivo Curvas_CSV/curvas_659.csv criado com sucesso
Arquivo Curvas_CSV/curvas_900.csv criado com sucesso
Arquivo Curvas_CSV/curvas_889.csv criado com sucesso
Arquivo Curvas_CSV/curvas_912.csv criado com sucesso
Arquivo Curvas_CSV/curvas_897.csv criado com sucesso
Arquivo Curvas_CSV/curvas_910.csv criado com sucesso
Arquivo Curvas_CSV/curvas_907.csv criado com sucesso


## Concatena os arquivos CSV das curvas em um único arquivo

In [13]:
import os

pasta = "Curvas_CSV"

# Lista para armazenar os dataframes repectivos aos .csv
df_list = []

# Itera sobre a pasta Curvas_CSV
for arquivo in os.listdir(pasta):
    file_path = os.path.join(pasta, arquivo)

    # Lê o arquivo CSV como um DF
    df = pd.read_csv(file_path)
    df_list.append(df)

# Concatena os DFs ao longo do eixo das colunas
df_concat = pd.concat(df_list, axis=0, ignore_index=True)

# Salva o DF concatenado em um .csv
df_concat.to_csv("curvas_pocos.csv", index=False)

print("Arquivo curvas_pocos.csv criado com sucesso")

Arquivo curvas_pocos.csv criado com sucesso


## Cria dataframes para os poços

In [14]:
c = dli_dict['574'].frames[0].curves()
type(c)

numpy.ndarray

In [15]:
dlis_df_dict = {}   # Conterá os dataframes respectivos aos poços

# Curvas de perfis escolhidas
curvas_escolhidas = ['TDEP', 'GR', 'NPHI', 'RHOB', 'DRHO', 'HDRS', 'LLD', 'BSZ', 'BS', 'CALI', 'DCALI', 'PE', 'DTC']

# Iterando sobre os arquivos lógicos de todos os poços (que estão armazenados em 'dli_dict'),
# '.values()' se refere aos valores do dicionário (não às chaves)
for chave, poco in dli_dict.items():

    # Armazenando as curvas que serão utilizadas em uma lista
    curvas_utilizadas = [
        channel.name                                    # Os elementos da lista serão os nomes das curvas
        for channel in poco.channels                    # As curvas são acessadas por meio de 'poco.channels'
        if channel.name in curvas_escolhidas            # As curvas que não utilizaremos não serão armazenadas na lista
    ]
    conjunto_aux = set(curvas_utilizadas)
    curvas_utilizadas_sem_duplicados = list(conjunto_aux)

    curvas = poco.frames[0].curves()

    # Criando um pandas dataframe do poço respectivo à atual iteração e armazenando o mesmo em dlis_df
    dlis_df_dict[chave] = pd.DataFrame(curvas[curvas_utilizadas_sem_duplicados])


In [16]:
dlis_df_dict.keys()

dict_keys(['551', '574', '595', '605', '659', '900', '889', '912', '897', '910', '907'])

## Transforma os valores -999.25 em nulos

In [17]:
for poco in dlis_df_dict.values():
    poco.replace([-999.25], [None], inplace = True)

## Aplicando os mnemônicos

In [18]:
aplica_mnemonico(dlis_df_dict, ['BS', 'BSZ'], 'BS')
aplica_mnemonico(dlis_df_dict, ['LLD', 'HDRS'], 'RESD')
aplica_mnemonico(dlis_df_dict, ['RHOB', 'RHOZ'], 'RHOB')

## Renomeia CALI para CAL

In [19]:
for poco in dlis_df_dict.values():
    renomeia_coluna(poco, 'CALI', 'CAL')

## Adiciona coluna DCAL

In [20]:
add_DCAL(dlis_df_dict)

## Preenche os poços com curvas faltando

In [21]:
# Se um dos poços não tiver uma dessas curvas, adicionamos a coluna da curva e mantemos os valores como None
curvas_obrigatorias = ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']

# Percorre todos os poços
for poco in dlis_df_dict.values():
    # Percorre todas as curvas obrigatórias
    for curva in curvas_obrigatorias:
        # Se o poço não tiver a curva
        if curva not in poco.columns:
            # Adiciona a coluna e os valores dela = None
            poco[curva] = None

## Temos as seguintes curvas 

In [22]:
for key, poco in dlis_df_dict.items():
    curvas = sorted(poco.keys())
    print(f"{key}: {curvas}")

551: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
574: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
595: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
605: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
659: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
900: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
889: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
912: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
897: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
910: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
907: ['BS', 'CAL', 'DCAL', 'DRHO', 'DTC', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']


## Remove valores DRHO e DCAL indesejados (Só depois)

In [23]:
#limita_curva(dlis_df_dict, "DRHO", -0.15, 0.15)
#limita_curva(dlis_df_dict, "DCAL", -2, 2)

## Inverte a ordem das linhas dos dataframes

In [24]:
for key in dlis_df_dict.keys():
    dlis_df_dict[key] = dlis_df_dict[key].iloc[::-1]

## Ordena os dataframes

In [31]:
ordem_desejada = ['TDEP', 'GR', 'RESD', 'BS', 'CAL', 'DCAL', 'NPHI', 'PE', 'DRHO', 'RHOB', 'DTC']

for key in dlis_df_dict.keys():
    dlis_df_dict[key] = dlis_df_dict[key].reindex(columns=ordem_desejada)

## Salva os dados dos dataframes em arquivos CSV

In [35]:
for key, value in dlis_df_dict.items():
    file_name = f"Pocos_CSV/poco_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

Arquivo Pocos_CSV/poco_551.csv criado com sucesso.
Arquivo Pocos_CSV/poco_574.csv criado com sucesso.
Arquivo Pocos_CSV/poco_595.csv criado com sucesso.
Arquivo Pocos_CSV/poco_605.csv criado com sucesso.
Arquivo Pocos_CSV/poco_659.csv criado com sucesso.
Arquivo Pocos_CSV/poco_900.csv criado com sucesso.
Arquivo Pocos_CSV/poco_889.csv criado com sucesso.
Arquivo Pocos_CSV/poco_912.csv criado com sucesso.
Arquivo Pocos_CSV/poco_897.csv criado com sucesso.
Arquivo Pocos_CSV/poco_910.csv criado com sucesso.
Arquivo Pocos_CSV/poco_907.csv criado com sucesso.
